# Key recovery attack on GGH-style signature scheme

In [1]:
from ggh.signature import keygen, sign, verify
from ggh.arithmetic import ortho_defect

n = 5
pk, sk, radius = keygen(n)

print(f"ortho-defect of sk is {ortho_defect(sk)}")
print(f"ortho-defect of pk is {ortho_defect(pk)}")

ortho-defect of sk is 2.1984871040703586
ortho-defect of pk is 754549602373544.9
